**FLAGS**

Modeli strojnog učenja često zahtjevaju da mnogi parametri budu zadani na početku modela, te da se ti isti parametri po mogućnosti lako mijenjaju, kao što je npr. broj neurona u neuronskoj mreži, *learning rate*, *batch size* itd. Korištenje zastavica (*flags*) je jednostavan način kako bi preko komandne linije mijenjali neki parameter u kodu. Prvi argument je ime zastavice, drugi je zadana vrijednost, a treći opis zastavice.

**MODEL**

Kao arhitektura modela izabrana je LSTM neuronska mreža, posebna vrsta rekurentnih neuronskih mreža koja je dobra za učenje dugotrajnih ovisnosti i pamti informacije u dužem periodu.
Neki od parametara koji su zadani su:

*silence_percentage* – povećavanjem će model raditi u korist *true positive* za *silence* i smanjenjem *false negatives* za riječi

*unknown_percentage* – zadano je da 10% podataka za treniranje bira iz *unknown* klase, povećavanjem će model manje vjerojatno zamijeniti uknown riječi za željene, ali ako je prevelik model će zaključiti da je najsigurnije sve riječi kategorizirati kao uknown

*learning_rate* – zadan je za određeni broj iteracija; kontrolira brzinu ažuriranja težina neuronske mreže

*background_volume*  - glasnoća; 0 je tišina a 1 dokraja pojačana glasnoća

*background_frequency* - u kojoj proporciji je background noise dodan u zvučne zapise 

*batch_size* - broj primjeraka iz uzorka za treniranje u jednoj iteraciji treniranja 

**TRENIRANJE**

Tijekom treniranja vidimo na kojem smo koraku treniranja. Za svakih deset koraka su nam pokazane *accuracy* (točnost) i *cross entropy* vrijednosti. Točnost  nam pokazuje koliko je klasa točno predviđeno na određenom koraku treniranja. Ovaj rezultat često varira, ali se u pravilu povećava kako treniranje mreže ide kraju. *Cross entropy* je rezultat funkcije gubitka, i vrijednost bi trebala padati tijekom treniranja. Nakon svakih 500 koraka dobivamo matricu konfuzije.

import sys
import os
import tensorflow as tf

sys.path.append("../libs")
from classification import input_data
from classification import models
from classification import trainer
from classification import freeze

tf.reset_default_graph()

In [2]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)





del_all_flags(tf.flags.FLAGS)

In [3]:

flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('data_dir','../data/raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','../summaries','Summaries Folder')
flags.DEFINE_string('models_dir','../models','Models Folder')
flags.DEFINE_string('wanted_words','yes,no,up,down,left,right,on,off,stop,go','Wanted Words')
flags.DEFINE_float('validation_percentage',10,'Validation Percentage')
flags.DEFINE_float('testing_percentage',10,'Testing Percentage')
flags.DEFINE_integer('sample_rate',16000,'Sample Rate')
flags.DEFINE_integer('clip_duration_ms',1000,'Clip Duration in ms')
flags.DEFINE_float('window_size_ms',40,'How long each spectogram timeslice is')
flags.DEFINE_float('window_stride_ms',20,'How far to move in time between frequency windows.')
flags.DEFINE_integer('dct_coefficient_count',40,'How many bins to use for the MFCC fingerprint')
flags.DEFINE_float('time_shift_ms',100.0,'Range to randomly shift the training audio by in time.')

FLAGS=flags.FLAGS

In [4]:
model_architecture='lstm_l'
logging_interval=10
eval_step_interval=500
save_step_interval=50000
silence_percentage=10.0
unknown_percentage=15.0
background_frequency=1
background_volume=0.3
learning_rate='0.0005,0.0001,0.00002,0.0001,0.00002' 
train_steps='10000,10000,10000,15000,5000' 
batch_size=256


In [5]:
tf.app.flags.DEFINE_string('f', '', 'kernel')
train_dir=os.path.join(FLAGS.data_dir,'train','audio')

In [6]:
model_settings = models.prepare_model_settings(
      len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
      FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
      FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
      train_dir, silence_percentage, unknown_percentage,
      FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
      FLAGS.testing_percentage, model_settings,use_silence_folder=True)

../data/raw\train\audio\*\*.wav
..\data\raw\train\audio\_background_noise_\doing_the_dishes.wav
..\data\raw\train\audio\_background_noise_\dude_miaowing.wav
..\data\raw\train\audio\_background_noise_\exercise_bike.wav
..\data\raw\train\audio\_background_noise_\pink_noise.wav
..\data\raw\train\audio\_background_noise_\running_tap.wav
..\data\raw\train\audio\_background_noise_\white_noise.wav
Tensor("AudioSpectrogram:0", shape=(?, 49, 513), dtype=float32)


In [7]:
def get_train_data(args):
    sess=args
    time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)
    train_fingerprints, train_ground_truth = audio_processor.get_data(
        batch_size, 0, model_settings,background_frequency,
        background_volume, time_shift_samples, 'training', sess)
    return train_fingerprints,train_ground_truth

In [8]:
def get_val_data(args):
    '''
    Input: (sess,offset)
    '''
    sess,i=args
    validation_fingerprints, validation_ground_truth = (
            audio_processor.get_data(batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
    return validation_fingerprints,validation_ground_truth

In [9]:
def get_test_data(args):
    '''
    Input: (sess,offset)
    '''
    sess,i=args
    test_fingerprints, test_ground_truth = audio_processor.get_data(
        batch_size, i, model_settings, 0.0, 0.0, 0, 'testing', sess)
    return test_fingerprints,test_ground_truth

In [10]:
def main(_):
    sess=tf.InteractiveSession()
    # Placeholders
    fingerprint_size = model_settings['fingerprint_size']
    label_count = model_settings['label_count']
    fingerprint_input = tf.placeholder(
      tf.float32, [None, fingerprint_size], name='fingerprint_input')
    ground_truth_input = tf.placeholder(
      tf.float32, [None, label_count], name='groundtruth_input')
    set_size = audio_processor.set_size('validation')
    label_count = model_settings['label_count']
    
    #Model
    
    logits, dropout_prob = models.create_model(
      fingerprint_input,
      model_settings,
      model_architecture,
      is_training=True)
    
    
    extra_args=(dropout_prob,label_count,batch_size,set_size)
    trainer.train(sess,logits,fingerprint_input,ground_truth_input,get_train_data,
                  get_val_data,train_steps,learning_rate,eval_step_interval, logging_interval=logging_interval,
                  checkpoint_interval=save_step_interval,
                  model_name=model_architecture,summaries_dir=FLAGS.summaries_dir,args=extra_args)

In [11]:
tf.app.run(main=main)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
INFO:tensorflow:Training from step: 1 
INFO:tensorflow:Step #10: rate 0.000500, accuracy 13.7%, cross entropy 2.418411
INFO:tensorflow:Step #20: rate 0.000500, accuracy 13.7%, cross entropy 2.378950
INFO:tensorflow:Step #30: rate 0.000500, accuracy 13.3%, cross entropy 2.275060
INFO:tensorflow:Step #40: rate 0.000500, accuracy 18.0%, cross entropy 2.271630
INFO:tensorflow:Step #50: rate 0.000500, accuracy 18.0%, cross entropy 2.247811
INFO:tensorflow:Step #60: rate 0.000500, accuracy 20.7%, cross entropy 2.124499
INFO:tensorflow:Step #70: rate 0.000500, accuracy 26.6%, cross entropy 2.069001
INFO:tensorflow:Step #80: rate 0.000500, accuracy 28.5%, cross entropy 2.055431
INFO:tensorflow:Step #90: rate 0.000500, accuracy 

INFO:tensorflow:Step #850: rate 0.000500, accuracy 73.0%, cross entropy 0.838817
INFO:tensorflow:Step #860: rate 0.000500, accuracy 72.7%, cross entropy 0.792592
INFO:tensorflow:Step #870: rate 0.000500, accuracy 69.5%, cross entropy 0.822250
INFO:tensorflow:Step #880: rate 0.000500, accuracy 73.4%, cross entropy 0.785683
INFO:tensorflow:Step #890: rate 0.000500, accuracy 68.0%, cross entropy 0.886864
INFO:tensorflow:Step #900: rate 0.000500, accuracy 77.0%, cross entropy 0.741588
INFO:tensorflow:Step #910: rate 0.000500, accuracy 74.6%, cross entropy 0.777758
INFO:tensorflow:Step #920: rate 0.000500, accuracy 66.8%, cross entropy 0.957895
INFO:tensorflow:Step #930: rate 0.000500, accuracy 71.1%, cross entropy 0.850936
INFO:tensorflow:Step #940: rate 0.000500, accuracy 73.4%, cross entropy 0.832193
INFO:tensorflow:Step #950: rate 0.000500, accuracy 72.3%, cross entropy 0.805397
INFO:tensorflow:Step #960: rate 0.000500, accuracy 73.4%, cross entropy 0.783174
INFO:tensorflow:Step #970: r

INFO:tensorflow:Step #1680: rate 0.000500, accuracy 80.1%, cross entropy 0.604877
INFO:tensorflow:Step #1690: rate 0.000500, accuracy 82.8%, cross entropy 0.591812
INFO:tensorflow:Step #1700: rate 0.000500, accuracy 77.7%, cross entropy 0.713535
INFO:tensorflow:Step #1710: rate 0.000500, accuracy 82.0%, cross entropy 0.503119
INFO:tensorflow:Step #1720: rate 0.000500, accuracy 78.5%, cross entropy 0.595545
INFO:tensorflow:Step #1730: rate 0.000500, accuracy 79.3%, cross entropy 0.527337
INFO:tensorflow:Step #1740: rate 0.000500, accuracy 80.1%, cross entropy 0.628776
INFO:tensorflow:Step #1750: rate 0.000500, accuracy 80.1%, cross entropy 0.597872
INFO:tensorflow:Step #1760: rate 0.000500, accuracy 75.8%, cross entropy 0.673825
INFO:tensorflow:Step #1770: rate 0.000500, accuracy 79.7%, cross entropy 0.628015
INFO:tensorflow:Step #1780: rate 0.000500, accuracy 76.2%, cross entropy 0.723255
INFO:tensorflow:Step #1790: rate 0.000500, accuracy 81.6%, cross entropy 0.511855
INFO:tensorflow:

INFO:tensorflow:Step #2510: rate 0.000500, accuracy 83.6%, cross entropy 0.520651
INFO:tensorflow:Step #2520: rate 0.000500, accuracy 79.3%, cross entropy 0.587951
INFO:tensorflow:Step #2530: rate 0.000500, accuracy 81.6%, cross entropy 0.588253
INFO:tensorflow:Step #2540: rate 0.000500, accuracy 85.2%, cross entropy 0.517748
INFO:tensorflow:Step #2550: rate 0.000500, accuracy 82.8%, cross entropy 0.495371
INFO:tensorflow:Step #2560: rate 0.000500, accuracy 78.9%, cross entropy 0.626962
INFO:tensorflow:Step #2570: rate 0.000500, accuracy 82.0%, cross entropy 0.535267
INFO:tensorflow:Step #2580: rate 0.000500, accuracy 79.7%, cross entropy 0.530213
INFO:tensorflow:Step #2590: rate 0.000500, accuracy 86.7%, cross entropy 0.409292
INFO:tensorflow:Step #2600: rate 0.000500, accuracy 81.2%, cross entropy 0.526605
INFO:tensorflow:Step #2610: rate 0.000500, accuracy 81.2%, cross entropy 0.479779
INFO:tensorflow:Step #2620: rate 0.000500, accuracy 84.8%, cross entropy 0.512703
INFO:tensorflow:

INFO:tensorflow:Step #3430: rate 0.000500, accuracy 82.4%, cross entropy 0.506546
INFO:tensorflow:Step #3440: rate 0.000500, accuracy 81.6%, cross entropy 0.511659
INFO:tensorflow:Step #3450: rate 0.000500, accuracy 84.4%, cross entropy 0.481048
INFO:tensorflow:Step #3460: rate 0.000500, accuracy 85.9%, cross entropy 0.444155
INFO:tensorflow:Step #3470: rate 0.000500, accuracy 82.8%, cross entropy 0.503401
INFO:tensorflow:Step #3480: rate 0.000500, accuracy 82.4%, cross entropy 0.496913
INFO:tensorflow:Step #3490: rate 0.000500, accuracy 85.2%, cross entropy 0.427055
INFO:tensorflow:Step #3500: rate 0.000500, accuracy 84.0%, cross entropy 0.505949
INFO:tensorflow:Confusion Matrix:
 [[  0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 302   6  19   4  11   2   6  16   7   4  10]
 [  0   4 243   5   0   3   4   0   0   0   0   2]
 [  0   8   0 249   1   7   1   0   0   0   0   4]
 [  0   7   0   1 215   1   2   0   3  11  14   6]
 [  0   1   1   8   0 247   0   0   0   0   2   5]
 [ 

...

INFO:tensorflow:Step #46830: rate 0.000020, accuracy 95.7%, cross entropy 0.151808
INFO:tensorflow:Step #46840: rate 0.000020, accuracy 92.6%, cross entropy 0.196171
INFO:tensorflow:Step #46850: rate 0.000020, accuracy 91.4%, cross entropy 0.197464
INFO:tensorflow:Step #46860: rate 0.000020, accuracy 90.6%, cross entropy 0.254192
INFO:tensorflow:Step #46870: rate 0.000020, accuracy 91.0%, cross entropy 0.251648
INFO:tensorflow:Step #46880: rate 0.000020, accuracy 93.0%, cross entropy 0.201916
INFO:tensorflow:Step #46890: rate 0.000020, accuracy 92.2%, cross entropy 0.207883
INFO:tensorflow:Step #46900: rate 0.000020, accuracy 94.9%, cross entropy 0.195092
INFO:tensorflow:Step #46910: rate 0.000020, accuracy 94.1%, cross entropy 0.178066
INFO:tensorflow:Step #46920: rate 0.000020, accuracy 93.4%, cross entropy 0.160971
INFO:tensorflow:Step #46930: rate 0.000020, accuracy 92.2%, cross entropy 0.236752
INFO:tensorflow:Step #46940: rate 0.000020, accuracy 92.6%, cross entropy 0.189259
INFO

INFO:tensorflow:Step #47650: rate 0.000020, accuracy 94.5%, cross entropy 0.156762
INFO:tensorflow:Step #47660: rate 0.000020, accuracy 93.8%, cross entropy 0.158743
INFO:tensorflow:Step #47670: rate 0.000020, accuracy 93.4%, cross entropy 0.185700
INFO:tensorflow:Step #47680: rate 0.000020, accuracy 94.1%, cross entropy 0.167218
INFO:tensorflow:Step #47690: rate 0.000020, accuracy 93.4%, cross entropy 0.188644
INFO:tensorflow:Step #47700: rate 0.000020, accuracy 95.7%, cross entropy 0.154615
INFO:tensorflow:Step #47710: rate 0.000020, accuracy 91.4%, cross entropy 0.226166
INFO:tensorflow:Step #47720: rate 0.000020, accuracy 92.6%, cross entropy 0.208918
INFO:tensorflow:Step #47730: rate 0.000020, accuracy 94.1%, cross entropy 0.169923
INFO:tensorflow:Step #47740: rate 0.000020, accuracy 92.6%, cross entropy 0.201024
INFO:tensorflow:Step #47750: rate 0.000020, accuracy 93.0%, cross entropy 0.176430
INFO:tensorflow:Step #47760: rate 0.000020, accuracy 94.1%, cross entropy 0.187266
INFO

INFO:tensorflow:Step 48500: Validation accuracy = 89.0% (N=2964)
INFO:tensorflow:Step #48510: rate 0.000020, accuracy 96.5%, cross entropy 0.104558
INFO:tensorflow:Step #48520: rate 0.000020, accuracy 91.4%, cross entropy 0.214003
INFO:tensorflow:Step #48530: rate 0.000020, accuracy 94.9%, cross entropy 0.142391
INFO:tensorflow:Step #48540: rate 0.000020, accuracy 93.8%, cross entropy 0.163075
INFO:tensorflow:Step #48550: rate 0.000020, accuracy 93.4%, cross entropy 0.178246
INFO:tensorflow:Step #48560: rate 0.000020, accuracy 95.3%, cross entropy 0.140290
INFO:tensorflow:Step #48570: rate 0.000020, accuracy 91.4%, cross entropy 0.294631
INFO:tensorflow:Step #48580: rate 0.000020, accuracy 94.5%, cross entropy 0.175694
INFO:tensorflow:Step #48590: rate 0.000020, accuracy 92.2%, cross entropy 0.203508
INFO:tensorflow:Step #48600: rate 0.000020, accuracy 94.9%, cross entropy 0.176591
INFO:tensorflow:Step #48610: rate 0.000020, accuracy 94.5%, cross entropy 0.169270
INFO:tensorflow:Step #

INFO:tensorflow:Step #49410: rate 0.000020, accuracy 92.2%, cross entropy 0.265703
INFO:tensorflow:Step #49420: rate 0.000020, accuracy 91.4%, cross entropy 0.243761
INFO:tensorflow:Step #49430: rate 0.000020, accuracy 92.6%, cross entropy 0.222590
INFO:tensorflow:Step #49440: rate 0.000020, accuracy 93.0%, cross entropy 0.205923
INFO:tensorflow:Step #49450: rate 0.000020, accuracy 93.8%, cross entropy 0.209119
INFO:tensorflow:Step #49460: rate 0.000020, accuracy 93.0%, cross entropy 0.197979
INFO:tensorflow:Step #49470: rate 0.000020, accuracy 95.7%, cross entropy 0.140304
INFO:tensorflow:Step #49480: rate 0.000020, accuracy 93.0%, cross entropy 0.200992
INFO:tensorflow:Step #49490: rate 0.000020, accuracy 90.6%, cross entropy 0.237526
INFO:tensorflow:Step #49500: rate 0.000020, accuracy 93.8%, cross entropy 0.206468
INFO:tensorflow:Confusion Matrix:
 [[  0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 320   3  15   4   3   3   5  17   5   0  12]
 [  0   1 239   8   3   1   6   1

TypeError: expected str, bytes or os.PathLike object, not NoneType